# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2781414248400,
 'week52high': 189.51,
 'week52low': 122.9,
 'week52highSplitAdjustOnly': 189.58,
 'week52lowSplitAdjustOnly': 119.7,
 'week52change': 0.381618280933941,
 'sharesOutstanding': 16862838009,
 'float': 0,
 'avg10Volume': 101876605,
 'avg30Volume': 100522472,
 'day200MovingAvg': 166.85,
 'day50MovingAvg': 174.18,
 'employees': 153306,
 'ttmEPS': 6.12,
 'ttmDividendRate': 0.8966766811871141,
 'dividendYield': 0.005434540734713232,
 'nextDividendDate': '',
 'exDividendDate': '2022-02-04',
 'nextEarningsDate': '2022-04-22',
 'peRatio': 27.17350662902604,
 'beta': 1.2711903516006466,
 'maxChangePercent': 64.7349436793776,
 'year5ChangePercent': 3.9362054885669595,
 'year2ChangePercent': 1.91207659971149,
 'year1ChangePercent': 0.380929288415367,
 'ytdChangePercent': -0.07787409573300616,
 'month6ChangePercent': 0.15098958750071007,
 'month3ChangePercent': -0.03373025151799344,
 'month1ChangePercent': -0.02078088175009027,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.380929288415367

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_dataframe = final_dataframe.append(
            pd.Series(
                    [
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A'
                    ], index = my_columns
                ), ignore_index = True
            )
        except KeyError:
            pass
            
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,143.77,0.144066,N/A
1,AAL,16.62,-0.330736,N/A
2,AAP,210.08,0.136583,N/A
3,AAPL,167.35,0.37682,N/A
4,ABBV,162.50,0.63938,N/A
...,...,...,...,...
497,YUM,122.37,0.155186,N/A
498,ZBH,122.31,-0.244407,N/A
499,ZBRA,436.52,-0.051634,N/A
500,ZION,68.75,0.237211,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True) # inplace modifies the final_dataframe
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,80.34,2.3659,N/A
1,148,DVN,61.78,1.77758,N/A
2,316,MRO,25.34,1.15809,N/A
3,298,MCHP,79.97,1.13295,N/A
4,344,NVDA,269.65,1.08249,N/A
5,356,OXY,63.01,1.0566,N/A
6,42,APA,42.62,1.03735,N/A
7,89,CF,105.87,1.03309,N/A
8,109,COP,103.50,0.99032,N/A
9,343,NUE,147.96,0.953696,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    while True:
        portfolio_size = input('Enter the size of your portfolio: ')
        try:
            portfolio_size = float(portfolio_size)
            break
        except ValueError:
            print('Please enter a number!')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000
10000000.0


In [9]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

/mnt/c/Users/yashr/Desktop/code/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,274,LB,80.34,2.3659,2489
1,148,DVN,61.78,1.77758,3237
2,316,MRO,25.34,1.15809,7892
3,298,MCHP,79.97,1.13295,2500
4,344,NVDA,269.65,1.08249,741
5,356,OXY,63.01,1.0566,3174
6,42,APA,42.62,1.03735,4692
7,89,CF,105.87,1.03309,1889
8,109,COP,103.50,0.99032,1932
9,343,NUE,147.96,0.953696,1351


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [21]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        except KeyError:
            pass
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.72,N/A,0.146498,N/A,-0.191045,N/A,-0.0768002,N/A,0.0560783,N/A,N/A
1,AAL,16.24,N/A,-0.334685,N/A,-0.176228,N/A,-0.0323558,N/A,-0.0647945,N/A,N/A
2,AAP,209.52,N/A,0.137976,N/A,0.0255013,N/A,-0.0924978,N/A,-0.0510015,N/A,N/A
3,AAPL,171.35,N/A,0.381267,N/A,0.156515,N/A,-0.0336621,N/A,-0.0204427,N/A,N/A
4,ABBV,164.10,N/A,0.633207,N/A,0.533113,N/A,0.229499,N/A,0.107325,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.78,N/A,0.149126,N/A,-0.0206379,N/A,-0.0694347,N/A,-0.0316879,N/A,N/A
498,ZBH,125.76,N/A,-0.250583,N/A,-0.161473,N/A,0.01609,N/A,-0.00210271,N/A,N/A
499,ZBRA,439.66,N/A,-0.0508888,N/A,-0.210417,N/A,-0.243153,N/A,0.0404684,N/A,N/A
500,ZION,68.68,N/A,0.237531,N/A,0.237612,N/A,0.152041,N/A,-0.0419677,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [22]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
    # print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.72,N/A,0.146498,0.521912,-0.191045,0.0677291,-0.0768002,0.266932,0.0560783,0.683267,N/A
1,AAL,16.24,N/A,-0.334685,0.0239044,-0.176228,0.0816733,-0.0323558,0.394422,-0.0647945,0.119522,N/A
2,AAP,209.52,N/A,0.137976,0.511952,0.0255013,0.492032,-0.0924978,0.231076,-0.0510015,0.163347,N/A
3,AAPL,171.35,N/A,0.381267,0.858566,0.156515,0.74502,-0.0336621,0.386454,-0.0204427,0.26494,N/A
4,ABBV,164.10,N/A,0.633207,0.958167,0.533113,0.966135,0.229499,0.910359,0.107325,0.880478,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.78,N/A,0.149126,0.525896,-0.0206379,0.360558,-0.0694347,0.288845,-0.0316879,0.217131,N/A
498,ZBH,125.76,N/A,-0.250583,0.0557769,-0.161473,0.101594,0.01609,0.527888,-0.00210271,0.366534,N/A
499,ZBRA,439.66,N/A,-0.0508888,0.213147,-0.210417,0.0537849,-0.243153,0.0159363,0.0404684,0.563745,N/A
500,ZION,68.68,N/A,0.237531,0.687251,0.237612,0.85259,0.152041,0.830677,-0.0419677,0.189243,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [24]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,143.72,N/A,0.146498,0.521912,-0.191045,0.0677291,-0.0768002,0.266932,0.0560783,0.683267,0.38496
1,AAL,16.24,N/A,-0.334685,0.0239044,-0.176228,0.0816733,-0.0323558,0.394422,-0.0647945,0.119522,0.15488
2,AAP,209.52,N/A,0.137976,0.511952,0.0255013,0.492032,-0.0924978,0.231076,-0.0510015,0.163347,0.349602
3,AAPL,171.35,N/A,0.381267,0.858566,0.156515,0.74502,-0.0336621,0.386454,-0.0204427,0.26494,0.563745
4,ABBV,164.10,N/A,0.633207,0.958167,0.533113,0.966135,0.229499,0.910359,0.107325,0.880478,0.928785
...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,123.78,N/A,0.149126,0.525896,-0.0206379,0.360558,-0.0694347,0.288845,-0.0316879,0.217131,0.348108
498,ZBH,125.76,N/A,-0.250583,0.0557769,-0.161473,0.101594,0.01609,0.527888,-0.00210271,0.366534,0.262948
499,ZBRA,439.66,N/A,-0.0508888,0.213147,-0.210417,0.0537849,-0.243153,0.0159363,0.0404684,0.563745,0.211653
500,ZION,68.68,N/A,0.237531,0.687251,0.237612,0.85259,0.152041,0.830677,-0.0419677,0.189243,0.63994


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [25]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,63.18,N/A,1.04339,0.988048,1.2227,1,1.09747,1,0.438922,1,0.997012
1,MOS,67.50,N/A,0.900265,0.98008,0.944905,0.988048,0.763997,0.998008,0.393108,0.996016,0.990538
2,APA,41.84,N/A,1.02802,0.986056,1.13159,0.996016,0.65118,0.994024,0.221698,0.984064,0.99004
3,CF,102.27,N/A,1.0527,0.99004,1.03629,0.992032,0.508529,0.984064,0.309649,0.994024,0.99004
4,CVX,168.83,N/A,0.65318,0.964143,0.760775,0.978088,0.44568,0.98008,0.220554,0.982072,0.976096
5,BKR,36.51,N/A,0.609265,0.950199,0.581743,0.97012,0.563522,0.99004,0.219852,0.98008,0.97261
6,HAL,38.78,N/A,0.685415,0.968127,0.862476,0.986056,0.681411,0.996016,0.122376,0.926295,0.969124
7,COP,104.84,N/A,0.967278,0.984064,0.799379,0.982072,0.485003,0.982072,0.113982,0.900398,0.962151
8,KR,56.79,N/A,0.626578,0.956175,0.410534,0.948207,0.234912,0.918327,0.228802,0.986056,0.952191
9,DVN,61.61,N/A,1.76597,0.998008,1.19342,0.998008,0.560071,0.988048,0.0846966,0.814741,0.949701


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [26]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [40]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,OXY,63.18,316,1.04339,0.988048,1.2227,1,1.09747,1,0.438922,1,0.997012
1,MOS,67.50,296,0.900265,0.98008,0.944905,0.988048,0.763997,0.998008,0.393108,0.996016,0.990538
2,APA,41.84,478,1.02802,0.986056,1.13159,0.996016,0.65118,0.994024,0.221698,0.984064,0.99004
3,CF,102.27,195,1.0527,0.99004,1.03629,0.992032,0.508529,0.984064,0.309649,0.994024,0.99004
4,CVX,168.83,118,0.65318,0.964143,0.760775,0.978088,0.44568,0.98008,0.220554,0.982072,0.976096
5,BKR,36.51,547,0.609265,0.950199,0.581743,0.97012,0.563522,0.99004,0.219852,0.98008,0.97261
6,HAL,38.78,515,0.685415,0.968127,0.862476,0.986056,0.681411,0.996016,0.122376,0.926295,0.969124
7,COP,104.84,190,0.967278,0.984064,0.799379,0.982072,0.485003,0.982072,0.113982,0.900398,0.962151
8,KR,56.79,352,0.626578,0.956175,0.410534,0.948207,0.234912,0.918327,0.228802,0.986056,0.952191
9,DVN,61.61,324,1.76597,0.998008,1.19342,0.998008,0.560071,0.988048,0.0846966,0.814741,0.949701


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [41]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name="Momentum Strategy", index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [43]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [44]:
writer.save()